<h1 align="center">Programación Científica en Python</h1>
<h3 align="center">Cellular Automaton: Conway's Game of Life</h3>
<h6 align="center">Sebastián Bórquez González - sborquez@alumnos.inf.utfsm.cl</h6>

## Cellular Automaton: Conway's Game of Life

El _Juego de la Vida_ es una aplicación de autómatas celulares (conjunto de reglas), para simular la formación de patrones en el crecimiento de colonias de organismos biológicos.


Este juego se representa por medio de un arreglo bi-dimensional de __células vivas__ y __células muertas__. Las reglas para pasar de una generación a la otras son las siguientes (_Existen diferentes variaciones, pero estas son las más comunes_):

* __Sobrepoblación__: Si una célula viva es rodeada por más de tres células vivas, muere.
* __Estasis__: Si una célula viva es rodeada por dos o tres células vivas, sobrevive.
* __Subpoblación__: Si una célula viva es rodeada por menos de dos células vivas, muere.
* __Reproduction__: Si una célula muerta es rodeada por exáctamente tres células vivas, esta se vuelve una célula viva.

Aquí cada célula es representada como un píxel en una grilla/arreglo bi-dimensional.

Para más información visitar los siguientes links:
* [https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life](https://en.wikipedia.org/wiki/Conway%27s_Game_of_Life)
* [https://jakevdp.github.io/blog/2013/08/07/conways-game-of-life/](https://jakevdp.github.io/blog/2013/08/07/conways-game-of-life/)
* [https://bitstorm.org/gameoflife/](https://bitstorm.org/gameoflife/)

# Implementación

El mapa consiste de un arreglo bidimensional de booleans con bordes. El valor de cada celda representa si la célula se encuentra viva o muerta.

<img src="./mapa.png", width=360, height=300>

Para determinar el siguiente estado, a cada celda se le calcula su siguiente estado utilizando una **regla**.

Una **regla** consiste de un arreglo bidimensional de tamaño 3x3, y un conjunto de valores. Realizaremos convolución y si el resultado se encuentra en el conjunto de valores, esta se considera viva para la siguiente iteración, en caso contrario, es una célula muerta.

<img src="./regla.png", width=360, height=300>



In [1]:
from ipywidgets import interact, fixed, IntSlider

%matplotlib inline
%load_ext line_profiler
%load_ext memory_profiler

In [2]:
%%writefile game_of_life.py
import numba
import numpy as np
import matplotlib.pyplot as plt

def init_universe(rows, cols, cells):
    universe = np.zeros((rows+2, cols+2), bool)
    universe[cells[0],cells[1]] = True
    return universe

def init_universe_random(rows, cols, cells):
    universe = np.zeros((rows+2, cols+2), bool)
    r = np.random.randint(1,rows+1,cells)
    c = np.random.randint(1,cols+1,cells)
    universe[r,c] = True
    return universe

def show(universe):
    rows,cols = universe.shape
    plt.figure(figsize=(10,10))
    plt.imshow(universe[1:rows-1,1:cols-1], cmap='bwr')
    plt.axis('off')
    plt.grid()
    plt.show()

def nn_step(universe, new_universe, rule):
    rows, cols = universe.shape
    for i in range(1,rows-1):
        for j in range(1,cols-1):
            new_universe[i,j] = np.sum(universe[i-1:i+2,j-1:j+2] * rule)
    
    return new_universe


def nn_evolve(universe, rule, num, t):
    t0 = 0
    rows, cols = universe.shape
    new_universe = np.zeros((rows,cols), int)
    while t0 < t:
        new_universe = step(universe, new_universe, rule)
        universe = np.zeros((rows,cols), bool)
        for value in num:
            universe = np.logical_or(new_universe == value, universe)
        t0 += 1
    return universe 

@numba.jit('int64[:,:] (boolean[:,:], int64[:,:], int64[:,:])', nopython=True)
def step(universe, new_universe, rule):
    rows, cols = universe.shape
    for i in range(1,rows-1):
        for j in range(1,cols-1):
            new_universe[i,j] = np.sum(universe[i-1:i+2,j-1:j+2] * rule)
    
    return new_universe

@numba.jit('boolean[:,:] (boolean[:,:], int64[:,:], int64[:,:], int64)')
def evolve(universe, rule, num, t):
    t0 = 0
    rows, cols = universe.shape
    new_universe = np.zeros((rows,cols), int)
    while t0 < t:
        new_universe = step(universe, new_universe, rule)
        universe = np.zeros((rows,cols), bool)
        for value in num:
            universe = np.logical_or(new_universe == value, universe)
        t0 += 1
    return universe 

rules = {
    "Standard" : (np.array([[1,1,1],[1,-9,1],[1,1,1]], int), np.array([3,-6,-7], int)),
    "Diagonales" : (np.array([[1,0,1],[0,-9,0],[1,0,1]], int), np.array([3,-6,-7], int)),
    "Cruz" : (np.array([[0,1,0],[1,-9,1],[0,1,0]], int), np.array([3,-6,-7], int)),
    "Fast Grow": (np.array([[1,1,1],[1,-9,1],[1,1,1]], int), np.array([3,4,5,6,7,-5,-6,-7,-8], int)),
    "Strong": (np.array([[1,1,1],[1,-9,1],[1,1,1]], int), np.array([3,4,-5,-6,-7,-8], int)),
    "xD": (np.array([[1,1,1],[1,-9,1],[1,1,1]], int), np.array([1,8, -1], int))
}

Overwriting game_of_life.py


In [3]:
from game_of_life import *

# Profiling

In [4]:
universe = init_universe_random(250,250,2000)
regla = rules["Standard"][0]
valores = rules["Standard"][1]

### timeit para diferentes iteracione

#### Sin numba

In [ ]:
%timeit nn_evolve(universe, regla, valores, 10)

In [ ]:
%timeit nn_evolve(universe, regla, valores, 500)

#### Con numba

In [ ]:
%timeit evolve(universe, regla, valores, 10)

In [ ]:
%timeit evolve(universe, regla, valores, 500)

In [ ]:
%timeit evolve(universe, regla, valores, 1000)

### Line Profiling


In [ ]:
%lprun -T lprof0 -f nn_evolve nn_evolve(universe, regla, valores, 100)

In [ ]:
print(open('lprof0', 'r').read())

In [ ]:
%lprun -T lprof1 -f evolve evolve(universe, regla, valores, 100)

In [ ]:
print(open('lprof1', 'r').read())

### Memory Profiling

In [ ]:
%mprun -T mprof1 -f evolve evolve(universe, regla, valores, 100)

In [ ]:
print(open('mprof1', 'r').read())

# Visualización


In [ ]:
universe = init_universe_random(100,100,5000)

@interact(universe=fixed(universe), rule=rules, t=IntSlider(min=0,max=500,step=1,value=0))
def evolution(universe, rule, t):
    show(evolve(universe, rule[0], rule[1], t))
